torch.Size([2, 6, 3])


In [ ]:
import torch
import torch.nn as nn

inputs=torch.tensor(  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) #
class maskedattention(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_query=nn.Linear(3,3,bias=False)
        self.W_key=nn.Linear(3,3,bias=False)
        self.W_value=nn.Linear(3,3,bias=False)
        self.dropout=nn.Dropout(0.5)
        self.register_buffer('mask', torch.triu(torch.ones(6, 6), diagonal=1)) # New
    def forward(self,inputs):
        b,context_length,dim=inputs.shape
        key=self.W_key(inputs)
        value=self.W_value(inputs)
        query=self.W_query(inputs)
        attention_score=query@key.transpose(1,2)
        attention_score.masked_fill(self.mask.bool()[:context_length,:context_length],-torch.inf)
        attn_weights = torch.softmax(
            attention_score / key.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New
        context_vector=attn_weights@value
        return context_vector
maskedAttention=maskedattention()
torch.manual_seed(123)
maskedAttention(batch)

torch.Size([2, 6, 3])


tensor([[[0.3774, 0.2591, 0.1462],
         [0.3732, 0.2517, 0.1517],
         [0.3732, 0.2517, 0.1517],
         [0.3735, 0.2527, 0.1500],
         [0.3738, 0.2526, 0.1514],
         [0.3734, 0.2527, 0.1496]],

        [[0.3774, 0.2591, 0.1462],
         [0.3732, 0.2517, 0.1517],
         [0.3732, 0.2517, 0.1517],
         [0.3735, 0.2527, 0.1500],
         [0.3738, 0.2526, 0.1514],
         [0.3734, 0.2527, 0.1496]]], grad_fn=<UnsafeViewBackward0>)